# 16. Tutorial 8: Basic econometric analysis (3)

#### Naoki TANI
#### Center for Advanced Policy Studies (CAPS), Institute of Economic Research, Kyoto University
#### May 30, 2024

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from statsmodels.formula.api import ols, logit, probit
from statsmodels.iolib.summary2 import summary_col


KeyboardInterrupt



## 1. Difference-in-Differences (DiD)
### 1-1. Introduction
#### DiD is one of the most common approaches for analyzing the causal effect of policy intervention.
#### We start with the canonical version of DiD with $2$ periods and $2$ groups.
#### ・2 periods: `t` (before the intervention) and `t+1` (after the intervention).
#### ・2 groups: `treated` and `untreated`.

#### Suppose that
#### ・$Y_{treated,t}$ is treated group's outcome before treatment period.
#### ・$Y_{untreated,t}$ is untreated group's outcome before treatment period.
#### ・$Y_{untreated,t+1}$ is untreated group's outcome after treatment period.
#### ・$Y_{treated,t+1}(1)$ is treated group's treated outcome after treatment period.
#### ・$Y_{treated,t+1}(0)$ is treated group's untreated outcome after treatment period (unobserved).

#### The average treatment effect is
#### $$\text{ATE} = Y_{treated,t+1}(1) - Y_{treated,t+1}(0),$$
#### but we do not observe $Y_{treated,t+1}(0)$.
#### We estimate it by using parallel trends assumption.
#### $$\widehat{\text{ATE}} = (Y_{treated,t+1}(1) - Y_{treated,t}) - (Y_{untreated,t+1} - Y_{untreated,t})$$

#### We can obtain $\widehat{\text{ATE}}$ by estimating $\delta$ of the linear regression model.
#### $$Y_{it} = \alpha + \beta \text{Treated_group}_i + \gamma \text{Treated_period}_t + \delta \text{Treated_group}_i \times \text{Treated_period}_it + \epsilon_{it}$$

In [ ]:
## Example of consumption data
establishment = pd.read_csv('R3census_b1_006_1.csv')
establishment.drop(index=[0,1,2,3,4], inplace=True)

# replace values of some cells in order to use the column of index "0" as a new column index. 
establishment.iloc[0,:5] = establishment.iloc[2,:5]
establishment.set_axis(establishment.iloc[0,:], axis=1, inplace=True)
establishment

establishment.drop(index=[5,6,7], inplace=True)

# Extract relevant columns
establishment = establishment.loc[:,['地域区分','産業大分類','産業中分類','事業所数']]

#establishment['地域区分'].str.split('_', expand=True)
establishment = pd.concat([establishment, establishment['地域区分'].str.split('_', expand=True)], axis=1)
establishment.rename(columns={0: 'city_code', 1: 'city'}, inplace=True)
establishment.drop('地域区分',axis=1,inplace=True)

# convert data type
establishment['city_code'] = establishment['city_code'].astype('int64')

# Convert object to integer
establishment['事業所数'] = establishment['事業所数'].str.replace(',', '')
establishment['事業所数'] = establishment['事業所数'].replace('-','0').astype('int64')

# We want to get data of cities in `高知県`, `高松市`, `徳島市`, and `松山市`. 
establishment = establishment.loc[((establishment['city_code']>=39201) & (establishment['city_code']<=39428)) | #高知県内の市町村
                                  (establishment['city_code']==37201) | #高松市
                                  (establishment['city_code']==36201) | #徳島市
                                  (establishment['city_code']==38201)   #松山市
                                 ]

# Get number of establishments of "卸売業、小売業"
establishment = establishment.loc[establishment['産業大分類']=='I']

# Rename the column names
establishment.rename(columns={'事業所数':'num_establishments_retail'}, inplace=True)

# Drop unnecessary columns
establishment.drop(['産業大分類','産業中分類','city_code'], axis=1, inplace=True)

establishment;

In [ ]:
df = pd.read_csv('consumption_data.csv')

# drop rows if all values in the rows are NaN
df.dropna(how='all', inplace=True)

# rename column name
df.rename(columns={'決済日 の週':'week',
                 '業種①':'industry',
                 '性別':'sex',
                 '年代':'age',
                 '市区町村':'city',
                 '居住地方①':'residence',
                 '人数':'num_user',
                 '金額':'value',
                 '件数':'num_uses'}, inplace=True)

#import datatime module
from datetime import datetime
# convert "年月日" string to datetime by using datetime module.
df['week'] = df['week'].map(lambda x: datetime.strptime(x,'%Y年%m月%d日'))

# Add "year" column
df['year'] = 2023
df.loc[df['week']<pd.to_datetime('2020-1-1'), 'year'] = 2019

# Convert object to integer
df['value'] = df['value'].str.replace(',', '').astype(int)

# "秘匿項目"
df['num_user'] = df['num_user'].replace(0.0,np.nan)
df['num_uses'] = df['num_uses'].replace(0.0,np.nan)

# Correct wrong city name
df['city'] = df['city'].replace('中村市','四万十市')
df['city'] = df['city'].replace('香美郡赤岡町','香南市')
df['city'] = df['city'].replace('香美郡土佐山田町','香美市')
df['city'] = df['city'].replace('吾川郡伊野町','吾川郡いの町')

# Remove "~~郡" from city column
import re

df['city'] = df['city'].map(lambda x: re.sub('吾川郡','',x))
df['city'] = df['city'].map(lambda x: re.sub('安芸郡','',x))
df['city'] = df['city'].map(lambda x: re.sub('幡多郡','',x))
df['city'] = df['city'].map(lambda x: re.sub('高岡郡','',x))
df['city'] = df['city'].map(lambda x: re.sub('長岡郡','',x))

# combine 2 DataFrames
df = pd.merge(df, establishment)

# extract data in 2023
df = df.loc[df['year']==2023]

df;

In [ ]:
df_gr = df.groupby(['city','week','num_establishments_retail']).agg({'value': 'sum'})
df_gr

# Transform multi-index to single index
df_gr.reset_index(level=['week','city','num_establishments_retail'], inplace=True)

# extract `高知市`, `高松市`, `徳島市`, `松山市`
df_gr  = df_gr.loc[(df_gr['city']=='高知市') | (df_gr['city']=='高松市')| (df_gr['city']=='徳島市')| (df_gr['city']=='松山市')]

# extract March and Aprl
df_gr = df_gr.loc[(df_gr['week']>=pd.to_datetime('2023-3-5')) & (df_gr['week']<=pd.to_datetime('2023-4-29'))]

df_gr;

In [ ]:
pd.to_datetime('2023-3-5')

In [ ]:
figure, ax  = plt.subplots(dpi=140)
#vertical line
ax.axvline(pd.to_datetime('2023-4-2'), linestyle=':', color="gray", linewidth = 1)

ax.plot(df_gr.loc[df_gr['city']=='高知市']['week'], df_gr.loc[df_gr['city']=='高知市']['value'], color='red', label="高知市")
ax.plot(df_gr.loc[df_gr['city']=='徳島市']['week'], df_gr.loc[df_gr['city']=='徳島市']['value'], color='blue', label="徳島市")
ax.plot(df_gr.loc[df_gr['city']=='高松市']['week'], df_gr.loc[df_gr['city']=='高松市']['value'], color='orange', label="高松市")
ax.plot(df_gr.loc[df_gr['city']=='松山市']['week'], df_gr.loc[df_gr['city']=='松山市']['value'], color='green', label="松山市")
###########################################################################
ax.legend(bbox_to_anchor=(1.05, 0.5), prop = {'family' : 'MS Gothic'}, loc='center left') #place legend outside the plot
ax.set_ylabel(r'$F_{N}(q_{n}(p))-p$',fontsize=10)
ax.set_xlabel("高知市、高松市、徳島市、松山市の消費額", fontname = 'MS Gothic',fontsize=10)
ax.tick_params(labelsize=7)
ax.set_title("高知市、高松市、徳島市、松山市の消費額（2023年、3-4月）", fontname = 'MS Gothic')

In [ ]:
# extract two groups (高知市 and 高松市) and two periods (March and May)
df_gr  = df_gr.loc[(df_gr['city']=='高知市') | (df_gr['city']=='高松市')]

#treated group dummy
df_gr['treated_group'] = 0
df_gr.loc[df_gr['city']=='高知市', 'treated_group'] =1

#treated period dummy
df_gr['treated_period'] = 0
df_gr.loc[df_gr['week']>=pd.to_datetime('2023-4-1'), 'treated_period'] =1

#treated group and period dummy
df_gr['treated_group_period'] = 0
df_gr.loc[(df_gr['week']>=pd.to_datetime('2023-4-1')) & (df_gr['city']=='高知市'), 'treated_group_period'] =1

df_gr;

#### It is possible to specify aggregation methods.

### 2-4. Scatterplot

In [ ]:
figure, ax  = plt.subplots(dpi=120)
ax.scatter(df_gr['num_establishments_retail'], df_gr['value']/1000000)
###########################################################################
ax.legend(bbox_to_anchor=(1.05, 0.5), prop = {'family' : 'MS Gothic'}, loc='center left') #place legend outside the plot
#ax.set_xlim(0, 1000)
#ax.set_ylim(0, 1000)
ax.set_ylabel("消費額（百万円）", fontname = 'MS Gothic')
ax.set_xlabel("事業所数", fontname = 'MS Gothic')
#ax.set_xlabel("percentile of industrial group earnings distribution")
ax.set_title("卸売業・小売業の事業所数と消費額の関係", pad = 20, fontname = 'MS Gothic')
figure.subplots_adjust(top=1.05, bottom=0.15) 

In [ ]:
df_gr

In [ ]:
formula = 'value ~ 1 + treated_group + C(week) + treated_group_period'
model = ols(formula, data=df_gr).fit()

In [ ]:
res = summary_col(model, stars=True, float_format='%0.4f',
                           info_dict={'N': lambda x: "{0:d}".format(int(x.nobs))},drop_omitted=True)
print(res)